# Неделя 7: NLP

## Генерация текста с помощью модели **GPT**

https://www.kaggle.com/tuckerarrants/text-generation-with-huggingface-gpt2

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# !pip install -q transformers

In [6]:
import numpy as np
import pandas as pd
import re
import random

import torch
from tqdm.notebook import tqdm
import transformers
from torch.optim import AdamW

device = "cuda" if torch.cuda.is_available() else "cpu"


import textwrap

In [7]:
device

'cuda'

In [8]:
# Загружаем токенайзер модели
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

In [9]:
# Загружаем файл с текстом, на котором будем дообучать модель
import re

path_to_txt = "netlenka_texts.txt"
with open(path_to_txt, encoding="utf8") as f:
    text = f.read()

text = re.sub("\n{2,}", "\n", text)
print(text[:1000])

 Слушай Глеб у нас в театральном кружке парней совсем мало А тут спектакль На тему революции и гражданской войны Сто лет всетаки Мэр наш из бывших комуняк и очень трепетно относится к этой шняге Сам обещался быть Помоги а В массовке там матросиком побудешь огаОга блядь! Это мой дружок детства Жека с которым я по юности панковал мне такое предложение сделал Мол грит ты же всегда когда нас менты принимали такое представление устраивал что тебе весь райотдел аплодировалНу тут то Жека подзабыл кой чего Сначала этот самый состав райотдела мне наваливал хороших бурбулей а потом уж одаривал аплодисментами Но то дела старые Сейчас же я давно остепенился и моих революционных порывов хватает максимум на воспитания кота хорошим пинком или чтение по пьяни Манифеста дремлющей черепахе на панцире которой я заебенил граффити «Свобода или смерть!» Черепахе было все равно Не убегать ни подыхать она не собиралась поэтому носила эту надпись как часть интерьерного декораЛадно грю допустим я согласен Все о

In [10]:
# токенизируем текст
tokens = tokenizer.encode(text, add_special_tokens=True)
tokens = np.array(tokens)
print(len(tokens))
tokens[:10]

Token indices sequence length is longer than the specified maximum sequence length for this model (756335 > 2048). Running this sequence through the model will result in indexing errors


756335


array([19896, 20473,   329,   813,   282, 13388,   547,  8370,   582,
       19707])

In [11]:
# разбиваем на train и test

l = len(tokens) // 15
train = []
test = []
for i in range(15):
    if i % 5 > 0:
        train.extend(tokens[i * l : (i + 1) * l])
    else:
        test.extend(tokens[i * l : (i + 1) * l])
train = np.array(train)
test = np.array(test)

print(len(tokens), len(train), len(test))

756335 605064 151266


In [12]:
from transformers import GPT2LMHeadModel

# Эту модель просто подгружаем и не будем дообучать
model_init = GPT2LMHeadModel.from_pretrained(
    "sberbank-ai/rugpt3small_based_on_gpt2",
    output_attentions=False,
    output_hidden_states=False,
)


# Эту модель подгрузим и далее обучим
model = GPT2LMHeadModel.from_pretrained(
    "sberbank-ai/rugpt3small_based_on_gpt2",
    output_attentions=False,
    output_hidden_states=False,
)

model.to(device)
model_init.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50264, bias=False)
)

In [13]:
batch_size = 4
max_len = 256
epochs = 5

n_train = len(train) // (batch_size * max_len)
n_test = len(test) // (batch_size * max_len)
print(n_train, n_test)

# устанавливаем оптимизатор
optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)

# трансформеры с трудом обучаются, для них нужны разные способы повышения
# эффективности градиентного спуска
total_steps = n_train * epochs
scheduler = transformers.get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=3, num_training_steps=total_steps
)


# зададим точность, хотя ориентироваться будем на качество генерации
def accuracy(y_true, logits):
    return (
        torch.mean((y_true[1:] == torch.argmax(logits, dim=2)[:-1]).float())
        .detach()
        .cpu()
        .numpy()
    )

590 147


In [14]:
torch.cuda.empty_cache()

In [15]:
!nvidia-smi

Thu Sep 25 17:34:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3080        Off |   00000000:01:00.0  On |                  N/A |
| 53%   43C    P0            109W /  340W |    2427MiB /  10240MiB |      5%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [16]:
# готовим тензоры для обучения размера [batch_size, max_len]


def prep_tensors(x, i, batch_size=batch_size, max_len=max_len):
    batch_ids = x[i * batch_size * max_len : (i + 1) * batch_size * max_len]
    batch_ids = batch_ids.reshape(batch_size, max_len)
    batch_ids = torch.tensor(batch_ids).to(device)
    return batch_ids


# обучающий цикл
for epoch in range(1, epochs + 1):
    print(f"epoch {epoch}/{epochs} : training")

    train_loss = []
    train_acc = []
    model.train()
    pbar = tqdm(range(n_train))
    for i in pbar:
        batch_ids = prep_tensors(train, i)

        model.zero_grad()  # optimizer.zero_grad()
        loss, logits, _ = model(
            batch_ids, token_type_ids=None, labels=batch_ids
        ).values()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        train_loss.append(loss.item())
        train_acc.append(accuracy(batch_ids, logits))
        # print(logits)
        pbar.set_description(
            f"acc {np.mean(train_acc):.4f} loss {np.mean(train_loss):.4f}", refresh=True
        )

    print(f"epoch {epoch}/{epochs} : validation")
    model.eval()
    val_acc = []
    val_loss = []
    pbar = tqdm(range(n_test))
    for i in pbar:
        batch_ids = prep_tensors(test, i)
        with torch.no_grad():
            loss, logits, _ = model(
                batch_ids, token_type_ids=None, labels=batch_ids
            ).values()

        val_loss.append(loss.item())
        val_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(
            f"acc {np.mean(val_acc):.4f} loss {np.mean(val_loss):.4f}", refresh=True
        )

epoch 1/5 : training


  0%|          | 0/590 [00:00<?, ?it/s]

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


epoch 1/5 : validation


  0%|          | 0/147 [00:00<?, ?it/s]

...

  0%|          | 0/147 [00:00<?, ?it/s]

epoch 5/5 : training


  0%|          | 0/590 [00:00<?, ?it/s]

epoch 5/5 : validation


  0%|          | 0/147 [00:00<?, ?it/s]

In [ ]:
logits.shape

torch.Size([8, 256, 50264])

Применим модель, которую мы не дообучали: просто для понимания разницы между дообученной на собственных данных моделью и предобученной.

In [17]:
# https://huggingface.co/transformers/main_classes/model.html#transformers.generation_utils.GenerationMixin.generate
# модель без дообучения

# prompt – строка, которую примет на вход и продолжит
prompt = """Ведьма вышла из леса"""

# токенизируем строку
prompt = tokenizer.encode(prompt, return_tensors="pt").to(device)

# out будет содержать результаты генерации в виде списка
out = (
    model_init.generate(
        # входная строка
        input_ids=prompt,
        # максимальная длина генерируемой последовательности
        max_length=70,
        # num_beams
        num_beams=5,
        # применяем сэмплирование
        do_sample=True,
        # применяем температуру
        temperature=1.0,
        # топ слов по вероятности
        top_k=50,
        # топ слов по суммарной вероятности
        top_p=0.6,
        # сколько (постараться) не повторять n_gram подряд
        no_repeat_ngram_size=3,
        # сколько вернуть генераций
        num_return_sequences=3,
    )
    .cpu()
    .numpy()
)

# out содержит результаты

In [ ]:
# out

In [18]:
# декодируем и печатаем
for out_ in out:
    print(tokenizer.decode(out_))

Ведьма вышла из леса и, как ни в чем не бывало, продолжила свой путь.

— Вот и все, — сказала она. — Теперь, когда я знаю, что ты здесь, я хочу знать, где ты и что с тобой. Я хочу, чтобы ты знала, что я люблю тебя, и я хочу,
Ведьма вышла из леса и, как ни в чем не бывало, продолжила свой путь.

— Вот и все, — сказала она. — Теперь, когда я знаю, что ты здесь, я хочу знать, где ты и что с тобой. Я хочу, чтобы ты знала, что я люблю тебя. Я люблю тебя,
Ведьма вышла из леса и, как ни в чем не бывало, продолжила свой путь.

— Вот и все, — сказала она. — Теперь, когда я знаю, что ты здесь, я хочу знать, где ты и что с тобой. Я хочу, чтобы ты знала, что я люблю тебя. Я люблю тебя и


In [19]:
# дообученная модель
with torch.inference_mode():
    prompt = """Ведьма вышла из леса"""
    prompt = tokenizer.encode(prompt, return_tensors="pt").to(device)
    out = (
        model.generate(
            input_ids=prompt,
            max_length=70,
            num_beams=10,
            do_sample=True,
            temperature=1.0,
            top_k=500,
            top_p=0.75,
            no_repeat_ngram_size=3,
            num_return_sequences=3,
        )
        .cpu()
        .numpy()
    )
    for out_ in out:
        print(textwrap.fill(tokenizer.decode(out_), 40), end="\n------------------\n")

Ведьма вышла из леса и направилась в
сторону поляныГдето на середине пути она
споткнулась о поваленное дерево и чуть
не упала на землю Упав на колени она
принялась рыться в рюкзаке в поисках
съестных припасов И тут же наткнулась на
пустую консервную банку в которой
плавали какието ошмётки мяса
------------------
Ведьма вышла из леса и направилась в
сторону поляныГдето на середине пути она
споткнулась о поваленное дерево и чуть
не упала на землю Упав на колени она
принялась рыться в рюкзаке в поисках
съестных припасов И тут же наткнулась на
пустую консервную банку в которой
плавали какието ошмётки и
------------------
Ведьма вышла из леса и направилась в
сторону поляныГдето на середине пути она
споткнулась о поваленное дерево и чуть
не упала на землю Упав на колени она
принялась рыться в рюкзаке в поисках
съестных припасов И тут же наткнулась на
пустую консервную банку в которой
плавали какието ошмётки К
------------------


# Пример сохранения модели

In [ ]:
# Сохраняем веса обученной модели
torch.save(model.state_dict(), "model.pt")

# Задаем класс модели (уже в streamlit/tg_bot)
model_finetuned = GPT2LMHeadModel.from_pretrained(
    "sberbank-ai/rugpt3small_based_on_gpt2",
    output_attentions=False,
    output_hidden_states=False,
)

# Вешаем сохраненные веса на нашу модель
model_finetuned.load_state_dict(torch.load("model.pt"))

# -> <All keys matched successfully>

<All keys matched successfully>

In [ ]:
model.save_pretrained("model_kish.pt")